# NewsQA Loop
11/2/2023 \
Lixiao Yang

This notebook provides a loop for different chunk and overlap sizes using GPT4ALL (falcon model) and LangChain, a small sample of text files are selected from NewsQA dataset.

Due to the computing resource, the result is limited, please follow these steps for full ledge running:
1. Replace file_path into 'combined-newsqa-data-v1.json' (80.2 MB) - to compile the json file, follow the option 1 log and [Docker method from NewsQA](https://github.com/Maluuba/newsqa#recommended-docker-set-up)
2. Update chunk_size and overlap_percentage
3. Revise calculate_em() function

In [12]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings.gpt4all import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from langchain.chains import RetrievalQA
from collections import Counter
import numpy as np
from collections import defaultdict
import json
from pathlib import Path
from langchain.prompts import PromptTemplate

In [13]:
# file_path='./combined-newsqa-data-v1-small.json'
file_path='/Users/wk77/Library/CloudStorage/OneDrive-DrexelUniversity/Documents/data/combined-newsqa-data-v1.json'
data = json.loads(Path(file_path).read_text())

In [ ]:
# data

In [ ]:
# for story in data['data']:
#     print(story['text'])
#     print("\n--- End of story ---\n")

In [14]:
# Helper function to calculate Exact Match (EM) score
def calculate_em(predicted, actual):
    return int(predicted == actual)

# Function to calculate the token-wise F1 score for text answers
def calculate_token_f1(predicted, actual):
    predicted_tokens = predicted.split()
    actual_tokens = actual.split()
    common_tokens = Counter(predicted_tokens) & Counter(actual_tokens)
    num_same = sum(common_tokens.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(predicted_tokens)
    recall = 1.0 * num_same / len(actual_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# Helper function to extract answer ranges from the consensus field
def extract_ranges(consensus):
    if 's' in consensus and 'e' in consensus:
        return [(consensus['s'], consensus['e'])]
    return []


template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Also provide me the source for your answer. Explain how to get the answer step by step.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [9]:
import time
import logging
# logging.basicConfig(level=logging.WARNING)  # This will show only warnings and errors
logging.basicConfig(level=logging.ERROR)

# Parameters
max_stories = 100
# Define chunk sizes and overlap percentages
chunk_sizes = [200,400,800,1600]
# chunk_sizes = [25,50,75,100]
overlap_percentages = [0,0.1,0.2,0.3]  # Expressed as percentages (0.1 = 10%)

# Results storage
f1_results = defaultdict(list)
em_results = defaultdict(list)
text_results = []

# Initialize the language model and the QA chain
# llm = GPT4All(model="C:/Users/24075/AppData/Local/nomic.ai/GPT4All/ggml-model-gpt4all-falcon-q4_0.bin", max_tokens=2048)
llm = GPT4All(model="/Users/wk77/Library/CloudStorage/OneDrive-DrexelUniversity/Documents/data/gpt4all/models/gpt4all-falcon-q4_0.gguf", max_tokens=2048)

# The following code would iterate over the stories and questions to calculate the scores
start_time = time.time()
print(f"{start_time} Started.")

output_file_path = "results/scores_20231115.txt"

with open(output_file_path, 'w') as file:
    # instantiate embedding ONCE
    word_embed = GPT4AllEmbeddings()
    for chunk_size in chunk_sizes:
        print(f"\n{time.time()-start_time} Processing chunk size {chunk_size}:")
        last_time = time.time()
        for overlap_percentage in overlap_percentages:
            actual_overlap = int(chunk_size * overlap_percentage)
            print(f"\n{time.time()-start_time}\t{time.time()-last_time}\tOverlap [{overlap_percentage}] {actual_overlap}")
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=actual_overlap)
    
            i = 0
            for story in data['data']:
                i+=1
                if i>max_stories:
                    break
                now_time = time.time()
                print(f"\n{now_time-start_time}\t{now_time-last_time}\t\tstory {i}: ",end='')
                last_time = now_time
    
                all_splits = text_splitter.split_text(story['text'])
                # print(f"[after split]")
                vectorstore = Chroma.from_texts(texts=all_splits, embedding=word_embed)
                # print(f"[after vector store]")
                qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(), return_source_documents=True)
                # print(f"[after qa chain]")
    
                j = 0
                for question_data in story['questions']:
                    j+=1
                    # print(f"{time.time()-start_time}\t\t\tquestion {j}")
                    print('.', end='')
    
                    # TODO: embed query and perform similarity_search_by_vector() instead
                    question = question_data['q']
                    question_vector = word_embed.embed_query(question)
                    # docs = vectorstore.similarity_search(question)
                    docs = vectorstore.similarity_search_by_vector(question_vector)
                    answer_ranges = extract_ranges(question_data['consensus'])
                    
                    # Get the prediction from the model
                    result = qa_chain({"query": question})
                    
                    # Check if the predicted answer is in the expected format (string)
                    predicted_answer = result['result']
                    if isinstance(predicted_answer, dict):
                        # If it's a dictionary, you need to adapt this part of the code to extract the answer string
                        predicted_answer = predicted_answer.get('answer', '')  # Assuming 'answer' is the key for the answer string
                    elif not isinstance(predicted_answer, str):
                        # If the answer is not a string and not a dictionary, log an error or handle it appropriately
                        print(f"Unexpected format for predicted answer: {predicted_answer}")
                        continue  # Skip to the next question
                    actual_answer = story['text'][answer_ranges[0][0]:answer_ranges[0][1]] if answer_ranges else ""
                    
                    # If there is an actual answer, get it from the story text using the character ranges
                    if answer_ranges:
                        actual_answer = story['text'][answer_ranges[0][0]:answer_ranges[0][1]]
                    else:
                        actual_answer = ""
                    
                    # Calculate the scores
                    em_score = calculate_em(predicted_answer, actual_answer)
                    f1_score_value = calculate_token_f1(predicted_answer, actual_answer)
                    file.write(f"{chunk_size}\t{overlap_percentage}\t{i}\t{j}\t{f1_score_value:.4f}\t{em_score:.2f}\n")
    
                    # Store the scores
                    em_results[(chunk_size, overlap_percentage)].append(em_score)
                    f1_results[(chunk_size, overlap_percentage)].append(f1_score_value)
    
                # delete object for memory
                del qa_chain
                del vectorstore
                del all_splits
            # delete splitter instance
            del text_splitter

1699634644.5507572 Started.
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
0.8240687847137451 Processing chunk size 200:
0.8240828514099121	2.86102294921875e-06	Overlap [0] 0

0.8241188526153564	3.790855407714844e-05		story 1: .

INFO:backoff:Backing off send_request(...) for 0.6s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


........
19.00162982940674	18.177510976791382		story 2: .........
40.59973478317261	21.59810495376587		story 3: .........
60.192909717559814	19.593174934387207		story 4: ........
76.77773070335388	16.584820985794067		story 5: .........
97.80094480514526	21.023214101791382		story 6: .........
117.5081717967987	19.707226991653442		story 7: .........
137.98197674751282	20.47380495071411		story 8: ............
166.1960849761963	28.21410822868347		story 9: ............
194.5884690284729	28.39238405227661		story 10: .........
215.52787590026855	20.939406871795654		story 11: .........
236.84424781799316	21.31637191772461		story 12: .......
254.07731986045837	17.23307204246521		story 13: .........
273.1314618587494	19.054141998291016		story 14: .........
292.21424984931946	19.08278799057007		story 15: .........
312.23126196861267	20.017012119293213		story 16: .......
329.3724527359009	17.141190767288208		story 17: .........
348.8036768436432	19.43122410774231		story 18: .........
371.874601840

INFO:backoff:Backing off send_request(...) for 0.5s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


........
4110.368706703186	20.2812659740448		story 39: .........
4131.533372879028	21.164666175842285		story 40: .........
5104.2739107608795	972.7405378818512		story 41: .

...


199337.30224585533	626.7657248973846		story 18: .........
199355.75024986267	18.448004007339478		story 19: .........
199389.08391284943	33.33366298675537		story 20: .........
200378.30595588684	989.2220430374146		story 21: .

INFO:backoff:Backing off send_request(...) for 0.7s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


.......
200402.31181383133	24.005857944488525		story 22: .........
201033.55439686775	631.2425830364227		story 23: .

INFO:backoff:Backing off send_request(...) for 0.1s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


........
201052.67179083824	19.117393970489502		story 24: .........
201078.07277584076	25.4009850025177		story 25: ...........
201122.40081381798	44.32803797721863		story 26: .........
201144.54829072952	22.1474769115448		story 27: .........
201169.93310070038	25.384809970855713		story 28: .........
201205.22987866402	35.296777963638306		story 29: ...........................
201288.09169888496	82.86182022094727		story 30: .........
201318.2768509388	30.185152053833008		story 31: .........
201340.85190582275	22.57505488395691		story 32: ...........
201384.096878767	43.244972944259644		story 33: ............
201413.24182200432	29.144943237304688		story 34: ........
201434.8893456459	21.647523641586304		story 35: .........
201462.61574292183	27.726397275924683		story 36: .........
201485.1547987461	22.539055824279785		story 37: ............................
201602.099370718	116.94457197189331		story 38: .........
201633.04647564888	30.947104930877686		story 39: .........
201657.3719649315	

In [11]:
# Calculate the average F1 and EM scores for each configuration
for config, scores in f1_results.items():
    avg_f1 = np.mean(scores)
    avg_em = np.mean(em_results[config])
    f1_results[config] = avg_f1
    em_results[config] = avg_em
    print(f"Chunk size {config[0]} with overlap {config[1]}% - Average F1: {avg_f1:.3f}, EM: {avg_em:.2f}")

# Output the results
print(f1_results)
print(em_results)

Chunk size 200 with overlap 0% - Average F1: 0.096, EM: 0.00
Chunk size 200 with overlap 0.1% - Average F1: 0.081, EM: 0.00
Chunk size 200 with overlap 0.2% - Average F1: 0.072, EM: 0.00
Chunk size 200 with overlap 0.3% - Average F1: 0.066, EM: 0.00
Chunk size 400 with overlap 0% - Average F1: 0.070, EM: 0.00
Chunk size 400 with overlap 0.1% - Average F1: 0.070, EM: 0.00
Chunk size 400 with overlap 0.2% - Average F1: 0.069, EM: 0.00
Chunk size 400 with overlap 0.3% - Average F1: 0.068, EM: 0.00
Chunk size 800 with overlap 0% - Average F1: 0.070, EM: 0.00
Chunk size 800 with overlap 0.1% - Average F1: 0.070, EM: 0.00
Chunk size 800 with overlap 0.2% - Average F1: 0.070, EM: 0.00
Chunk size 800 with overlap 0.3% - Average F1: 0.071, EM: 0.00
Chunk size 1600 with overlap 0% - Average F1: 0.071, EM: 0.00
Chunk size 1600 with overlap 0.1% - Average F1: 0.071, EM: 0.00
Chunk size 1600 with overlap 0.2% - Average F1: 0.071, EM: 0.00
Chunk size 1600 with overlap 0.3% - Average F1: 0.072, EM: 0

In [ ]:
# # Print out the F1 and EM Results
# print("\nF1 and EM Results:")
# for config in f1_results:
#     chunk_size, overlap_percentage = config
#     print(f"Chunk Size: {chunk_size}, Overlap Percentage: {overlap_percentage}%, F1: {f1_results[config]}, EM: {em_results[config]}")